In [286]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout

from utils.constants import MAX_NB_VARIABLES, NB_CLASSES_LIST, MAX_TIMESTEPS_LIST
from utils.keras_utils import train_model, evaluate_model, set_trainable
from utils.layer_utils import AttentionLSTM

DATASET_INDEX = 0

MAX_TIMESTEPS = MAX_TIMESTEPS_LIST[DATASET_INDEX]
MAX_NB_VARIABLES = MAX_NB_VARIABLES[DATASET_INDEX]
NB_CLASS = NB_CLASSES_LIST[DATASET_INDEX]

TRAINABLE = True

def generate_model():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))

    x = Masking()(ip)
    x = LSTM(8)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(64, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 4, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(64, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model

def squeeze_excite_block(input):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor

    Returns: a keras tensor
    '''
    filters = input._keras_shape[-1] # channel_axis = -1 for TF

    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([input, se])
    return se

model = generate_model()

train_model(model, DATASET_INDEX, dataset_prefix='vehicle_data', epochs=30, batch_size=16)

evaluate_model(model, DATASET_INDEX, dataset_prefix='vehicle_data', batch_size=16)

model.save('/Users/yaofan29597/Desktop/UVA/course/Cloud Computing/project/code/MLSTM_FCN/test_model.h5')

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 2, 506)       0                                            
__________________________________________________________________________________________________
permute_3 (Permute)             (None, 506, 2)       0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 506, 64)      704         permute_3[0][0]                  
__________________________________________________________________________________________________
batch_normalization_7 (BatchNor (None, 506, 64)      256         conv1d_7[0][0]                   
____________________________________________________________________________________________

 - 8s - loss: 0.5557 - accuracy: 0.8110 - val_loss: 0.7725 - val_accuracy: 0.7112

Epoch 00011: loss improved from 0.57601 to 0.55574, saving model to ./weights
Epoch 12/30
 - 8s - loss: 0.5282 - accuracy: 0.8181 - val_loss: 0.7241 - val_accuracy: 0.7238

Epoch 00012: loss improved from 0.55574 to 0.52816, saving model to ./weights
Epoch 13/30
 - 8s - loss: 0.5090 - accuracy: 0.8273 - val_loss: 0.5717 - val_accuracy: 0.7995

Epoch 00013: loss improved from 0.52816 to 0.50901, saving model to ./weights
Epoch 14/30
 - 8s - loss: 0.5242 - accuracy: 0.8175 - val_loss: 0.5615 - val_accuracy: 0.7831

Epoch 00014: loss did not improve from 0.50901
Epoch 15/30
 - 9s - loss: 0.4662 - accuracy: 0.8462 - val_loss: 0.9058 - val_accuracy: 0.6494

Epoch 00015: loss improved from 0.50901 to 0.46622, saving model to ./weights
Epoch 16/30
 - 9s - loss: 0.4696 - accuracy: 0.8370 - val_loss: 0.5278 - val_accuracy: 0.8058

Epoch 00016: loss did not improve from 0.46622
Epoch 17/30
 - 10s - loss: 0.4927 - 

(0.28625473380088806, 7.359690446120042)

In [380]:
sess=tf.InteractiveSession()

tlab = tf.constant(np.ones(7), dtype=tf.float32)
opt = tf.placeholder(tf.float32,[3, 7])
real = tf.reduce_sum((tlab)*opt,1)
ll = tf.maximum(0.0, real)

sess.run(ll, feed_dict={opt:np.ones((3,7))})

array([7., 7., 7.], dtype=float32)

In [ ]:
model.

In [154]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from utils.generic_utils import load_dataset_at
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from utils.constants import MAX_NB_VARIABLES, MAX_TIMESTEPS_LIST

ckpt_path = '/Users/yaofan29597/Desktop/UVA/course/Cloud Computing/project/code/MLSTM_FCN/test_model.h5'

from keras.models import load_model
model = load_model(ckpt_path)

X_train, y_train, X_test, y_test, is_timeseries = load_dataset_at(0,fold_index=None,normalize_timeseries=False) 
X_test = pad_sequences(X_test, maxlen=MAX_NB_VARIABLES[0], padding='post', truncating='post')
y_test = to_categorical(y_test, len(np.unique(y_test)))


loss, accuracy = model.evaluate(X_test[:], y_test[:], batch_size=16)

loss
accuracy


tlab = tf.constant(np.zeros(7), dtype=tf.float32)
opt = tf.constant(np.zeros(3,7), dtype=tf.float32)
real = tf.reduce_sum((tlab)*opt,1)



Loading train / test dataset :  ../cut_data/ ../cut_data/
Finished processing train dataset..
Finished loading test dataset..

Number of train samples :  1847 Number of test samples :  793
Number of classes :  7
Sequence length :  506
793/793 [==============================] - 1s 1ms/step


0.5680130276673979

0.7856242060661316

In [521]:
import tensorflow as tf
import math
import numpy as np
from tqdm import tqdm

sess=tf.InteractiveSession()

'''
    Three Running Mode: whitebox/fake_blackbox/blackbox. 
    Select one of these 3 options as True to proceed.
'''
# whitebox, fake_blackbox, blackbox = True, False, False
whitebox, fake_blackbox, blackbox = False, True, False
# whitebox, fake_blackbox, blackbox = False, False, True
if whitebox:
    use_train_op = True
if fake_blackbox:
    use_train_op, use_grad_op = False, True
if blackbox:
    use_train_op, use_grad_op = False, False

    
'''
    Construct tf-Graph
''' 
CONST_LAMBDA = 1000
SUCCESS_ATTACK_PROB_THRES = 0.2
x = tf.placeholder(tf.float32,[None, 2, 506])
y = tf.placeholder(tf.float32,[None, 7])

# In whitebox attack, Var adv is updated thru train_op, while in blackbox attack, adv is updated manually. 
with tf.name_scope('attack'):
    if whitebox:
        adv = tf.Variable(tf.zeros([1, 2, 506]), name = "adv_pert")
    else:
        adv = tf.placeholder(tf.float32, [None, 2, 506])

# specify trainable variable
all_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
attack_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='attack')
trainable_vars = tf.trainable_variables()
for var in all_vars:
    if var not in attack_vars:
        trainable_vars.remove(var)

# compute loss
new_x = adv + x
output_x = model(x)
output = model(new_x)

l2dist = tf.reduce_sum(tf.square(adv), [1,2])

real = tf.reduce_sum(y * output, 1)
fake = tf.reduce_max((1 - y) * output, 1)
    
loss1 = CONST_LAMBDA * tf.maximum(-SUCCESS_ATTACK_PROB_THRES, real - fake)
loss2 = l2dist
loss_batch = loss1 + loss2
loss = tf.reduce_sum(loss_batch) # sum over all the batch samples

optimizer = tf.train.AdamOptimizer(0.1)

# replace train_op with manual designed grad_op
if use_train_op:
    train = optimizer.minimize(loss, var_list=trainable_vars)
if use_grad_op:
    grad_op = tf.gradients(loss, adv)

# initialize variables and load target model
sess.run(tf.global_variables_initializer())
model.load_weights(ckpt_path)


'''
    Perform attack
'''

success_count = 0
fail_count = 0
invalid_count = 0

for i in range(X_test.shape[0]):
    
    print("start attacking target", i, "...")
    
    mt = 0               # accumulator m_t in Adam
    vt = 0               # accumulator v_t in Adam
    beta1=0.9            # parameter beta_1 in Adam
    beta2=0.999          # parameter beta_2 in Adam
    learning_rate = 1e-1 # learning rate
    epsilon = 1e-8       # parameter epsilon in Adam
    h = 0.0001           # discretization constant when estimating numerical gradient
    batch_size = 1       # batch size
    MAX_ITE = 1000       # maximum number of iterations


    real_adv = np.zeros([1, 2, 506])   # initial adversarial perturbation, the trainable variable
    X = X_test[i:i+1]           # target sample X
    Y = y_test[i:i+1]           # target sample's lable Y
    
    # check if (X, Y) is a valid target 
    pred_y = model.predict(X)
    if sum((max(pred_y[0])==pred_y[0]) * Y[0]) == 0:
        print("not a valid target.")
        invalid_count += 1
        continue
        

    # when performing blackbox attack, we feed forward X3 = [X; X+h; X-h] in the same batch for estimating the gradient
    if blackbox:
        var = np.repeat(real_adv, batch_size*3, axis=0)
        X3 = np.repeat(X, 3, axis=0)
        Y3 = np.repeat(Y, 3, axis=0)


    # main loop for the optimization
    for epoch in tqdm(range(1, MAX_ITE+1)):

        if use_train_op: # apply train_op

            # whitebox attack

            sess.run(train, feed_dict={x:X,y:Y})
            adv1, output1, l2dist1, real1, fake1, loss1, new_x1 = sess.run([adv, output, l2dist, real, fake, loss, new_x], feed_dict={x:X,y:Y})
            print(l2dist1, adv1, X, new_x1)
        else: # apply self-implemented Adam

            # estimate gradient. 
            if use_grad_op: # For fake blackbox attack, just run grad_op.

                # fake blackbox attack

                true_grads, los, l2s, los1, los2, scores, scores_x, nx, adv1 = sess.run([grad_op, loss, l2dist, loss1, loss2, output, output_x, new_x, adv], feed_dict={adv: real_adv, x:X, y:Y})

                # clip the gradient to non-zero coordinates
                true_grads[0][0][0:2,sum(abs(X[0][0])>0):]=0

                grad = true_grads[0].reshape(-1)

            else: # For blackbox attack, apply 1-order discretization

                # blackbox attack   

                var_size = real_adv[0].size # (2, 506)
                # randomly choose a coordinate to compute partial gradient
                update_indice = np.random.choice(var_size, 1, replace=True) 

                # compute coordinate-perturbed input as a batch of size [batch_size*3, 2, 506]
                # var = [X; X+h; X-h], X.size = [batch_size, 2, 506]
                for i in range(batch_size):
                    var[batch_size * 1 + i].reshape(-1)[update_indice[0]] += h
                    var[batch_size * 2 + i].reshape(-1)[update_indice[0]] -= h

                los, l2s, los_b, scores, nx, adv1 = sess.run([loss, l2dist, loss_batch, output, new_x, adv], feed_dict={adv: var, x:X3, y:Y3})

                grad = np.zeros(real_adv.reshape(-1).shape)

                # grad(x) = [loss(X+he)-loss(X-he)] / (2h) 
                for i in range(batch_size):
                    grad[update_indice[0]] += los_b[batch_size * 1 + i]- los_b[batch_size * 2 + i]
                grad[update_indice[0]] /= 2*h

            # Adam update
            mt = beta1 * mt + (1 - beta1) * grad
            vt = beta2 * vt + (1 - beta2) * np.square(grad)
            corr = (math.sqrt(1 - beta2 ** epoch)) / (1 - beta1 ** epoch)

            m = real_adv.reshape(-1)
            m -= learning_rate * corr * (mt / (np.sqrt(vt) + epsilon))
            real_adv = m.reshape(real_adv.shape)

            if use_grad_op: 
    #             print(los1, los2, scores, scores_x)
                if epoch == MAX_ITE:
                    print("attack failed!")
                    fail_count += 1
                    break
                if sum((scores[0] == max(scores[0]))*Y[0])==0:
                    print("attack successed! with ite =", epoch)
                    success_count += 1
                    break

            else:
                print(l2s[[0,batch_size,2*batch_size]], los)



/Users/yaofan29597/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


'\n    Three Running Mode: whitebox/fake_blackbox/blackbox. \n    Select one of these 3 options as True to proceed.\n'

'\n    Construct tf-Graph\n'

KeyboardInterrupt: 